# Présentation du notebook

Ce notebook a été conçu pour transformer une collection de fichiers texte, chacun étiqueté par langue, en un fichier CSV contenant des paires de phrases segmentées.

Fonctions principales :
- `segment_sentences(text: str, language: str) -> list`: Segmente un texte donné en phrases selon la langue spécifiée en utilisant les modèles NLP appropriés.
- `create_csv(folder: str, folder_csv: str, num_pairs: int = 550) -> None`: Regroupe des paires de phrases par langue et les écrit dans un fichier CSV équilibré.

Le script suit les étapes suivantes :
1. **Initialisation des modèles linguistiques** :
   - Plusieurs modèles NLP (spacy et stanza) sont chargés pour traiter différentes langues : anglais, français, chinois, etc.

2. **Segmenter un texte en phrases** :
   - La fonction `segment_sentences` est utilisée pour diviser un texte en phrases en utilisant des modèles NLP spécifiques à chaque langue.

3. **Création du fichier CSV** :
   - La fonction `create_csv` lit les fichiers texte du dossier source, les segmente en phrases, regroupe les phrases par paires, et écrit ces paires dans un fichier CSV.
   - Les fichiers texte sont identifiés par leurs noms, qui suivent un format indiquant la langue.

4. **Équilibrage des données** :
   - Un nombre fixe de paires est sélectionné pour chaque langue afin d'équilibrer les données dans le CSV final.


In [1]:
import os
import pandas as pd
import spacy, stanza
import jupyter_black
from collections import defaultdict
from random import sample

In [2]:
jupyter_black.load() # Charge l'extension jupyter-black pour formater le code

## Création du csv

In [3]:
nlp_en = spacy.load("en_core_web_sm")  # English
nlp_es = spacy.load("es_core_news_sm")  # Spanish
nlp_de = spacy.load("de_core_news_sm")  # German
nlp_fr = spacy.load("fr_core_news_sm")  # French
nlp_ru = spacy.load("ru_core_news_sm")  # Russian
nlp_zh = spacy.load("zh_core_web_sm")  # Chinese
nlp_ja = spacy.load("ja_core_news_sm")  # Japanese
nlp_ko = spacy.load("ko_core_news_sm")  # Korean
nlp_ar = stanza.Pipeline(lang="ar")  # Arabe


def segment_sentences(text: str, language: str) -> list:
    """
    Segmenter un texte en phrases.

    Parameters:
    text (str): Le texte à segmenter.
    language (str): La langue du texte.

    Returns:
    List[str]: Une liste de phrases.
    """
    if language == "ar":
        doc = nlp_ar(text)
        return [sentence.text.strip() for sentence in doc.sentences]
    else:
        nlp = globals()[f"nlp_{language}"]
        doc = nlp(text)
        return [sent.text.strip() for sent in doc.sents]


def create_csv(folder: str, folder_csv: str, num_pairs: int = 550) -> None:
    """
    Crée un fichier CSV équilibré à partir des fichiers texte dans le dossier donné.
    Chaque ligne du fichier CSV contient une paire de phrases et la langue de ces phrases.
    Les fichiers texte doivent être nommés selon le format suivant : "langue_nom.txt".

    Parameters:
    folder (str): Le chemin du dossier contenant les fichiers texte.
    folder_csv (str): Le chemin du dossier où le fichier CSV sera enregistré.
    num_pairs (int): Nombre de paires de phrases par langue.

    Returns:
    None
    """
    data_by_language = defaultdict(list)

    for file in os.listdir(folder):
        if file.startswith("."):  # Cela ignore .DS_Store et d'autres fichiers cachés
            continue

        lang = file.split("_")[0]
        with open(os.path.join(folder, file), "r", encoding="utf-8") as f:
            text = f.read()
        if text:
            # Utiliser sent_tokenize pour diviser le texte en phrases
            sentences = segment_sentences(text, lang)
            # Ajouter des paires de phrases
            if len(sentences) > 1:
                for i in range(
                    0, len(sentences) - 1, 2
                ):  # S'arrêter à l'avant-dernière phrase pour éviter un index hors limites
                    data_by_language[lang].append(" ".join(sentences[i : i + 2]))
            else:
                data_by_language[lang].append(" ".join(sentences))

    balanced_data = []
    for lang, pairs in data_by_language.items():
        # Limiter à `num_pairs` paires en sélectionnant aléatoirement ou ajouter des paires vides si moins que nécessaire
        if len(pairs) > num_pairs:
            pairs = sample(pairs, num_pairs)
        balanced_data.extend((lang, pair) for pair in pairs)

    # Créer un DataFrame et sauvegarder en CSV
    df_balanced = pd.DataFrame(balanced_data, columns=["Label", "Text"])
    os.makedirs(folder_csv, exist_ok=True)
    df_balanced.to_csv(
        os.path.join(folder_csv, "data.csv"), index=False, encoding="utf-8"
    )

2024-05-12 10:38:19 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES
INFO:stanza:Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


2024-05-12 10:38:20 INFO: Downloaded file to /Users/perrine/stanza_resources/resources.json
INFO:stanza:Downloaded file to /Users/perrine/stanza_resources/resources.json
2024-05-12 10:38:21 INFO: Loading these models for language: ar (Arabic):
| Processor | Package       |
-----------------------------
| tokenize  | padt          |
| mwt       | padt          |
| pos       | padt_charlm   |
| lemma     | padt_nocharlm |
| depparse  | padt_charlm   |
| ner       | aqmar_charlm  |

INFO:stanza:Loading these models for language: ar (Arabic):
| Processor | Package       |
-----------------------------
| tokenize  | padt          |
| mwt       | padt          |
| pos       | padt_charlm   |
| lemma     | padt_nocharlm |
| depparse  | padt_charlm   |
| ner       | aqmar_charlm  |

2024-05-12 10:38:21 INFO: Using device: cpu
INFO:stanza:Using device: cpu
2024-05-12 10:38:21 INFO: Loading: tokenize
INFO:stanza:Loading: tokenize
2024-05-12 10:38:21 INFO: Loading: mwt
INFO:stanza:Loading: mwt
20

La fonction `create_csv` est conçue pour transformer une collection de fichiers textuels, chacun étiqueté par langue, en un fichier CSV contenant des paires de phrases. Voici les étapes détaillées du traitement implémenté par cette fonction :

1. **Initialisation des structures de données** :
   - Un dictionnaire `data_by_language` est initialisé avec `defaultdict(list)`. Ce dictionnaire va accumuler les phrases segmentées, regroupées par langue.

2. **Traitement des fichiers texte** :
   - La fonction parcourt tous les fichiers dans le dossier spécifié (`folder`). Elle ignore les fichiers cachés (comme `.DS_Store`).
   - Pour chaque fichier valide, la langue est déterminée à partir de la partie initiale du nom du fichier (séparée par `_`). Par exemple, un fichier nommé "fr_texte.txt" serait associé à la langue française (`fr`).
   - Chaque fichier est lu pour en extraire le texte intégral, et ce texte est ensuite segmenté en phrases à l'aide de la fonction `segment_sentences`, qui utilise des modèles de traitement du langage naturel spécifiques à chaque langue pour une segmentation précise.

3. **Construction de paires de phrases** :
   - Les phrases sont groupées par paires consécutives. Si le nombre total de phrases est impair, la dernière phrase reste seule. Ces paires (ou parfois une seule phrase) sont ensuite stockées dans le dictionnaire `data_by_language` sous la clé correspondante à la langue.

4. **Équilibrage et préparation des données pour le CSV** :
   - Après avoir collecté les paires de phrases pour toutes les langues, la fonction ajuste le nombre de paires pour chaque langue à `num_pairs`, en utilisant la fonction `sample` pour réduire ou maintenir le nombre de paires à ce seuil.
   - Un tableau de données est créé pour chaque langue, contenant les paires de phrases avec l'étiquette de langue correspondante.

5. **Création du DataFrame et enregistrement en CSV** :
   - Les données sont assemblées dans un DataFrame Pandas nommé `df_balanced` avec les colonnes "Label" pour la langue et "Text" pour les paires de phrases.
   - Le dossier de sortie (`folder_csv`) est vérifié et créé si nécessaire avec `os.makedirs`.
   - Le DataFrame est enregistré dans un fichier CSV nommé "data.csv" dans le dossier `folder_csv`, sans inclure les indices du DataFrame et en utilisant l'encodage UTF-8 pour assurer la compatibilité des caractères.


In [4]:
def main():
    folder_csv = "../../../data/csv/"
    folder_txt_files = "../../../data/clean/"
    create_csv(folder_txt_files, folder_csv)

    print("Fichier CSV créé avec succès.")

main()

Fichier CSV créé avec succès.
